In [2]:
import cv2
from PIL import Image


In [3]:
import numpy as np
import cv2

In [4]:

def get_limits(color):
    c = np.uint8([[color]])  # BGR values
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    hue = hsvC[0][0][0]  # Get the hue value

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit

In [13]:
yellow = [0, 255, 255]  # yellow in BGR colorspace
cap = cv2.VideoCapture(0)

In [15]:
blue_bgr = [255, 0, 0] 
cap = cv2.VideoCapture(0)


In [17]:
red_bgr = [0, 0, 255]  # BGR for red
cap = cv2.VideoCapture(0)


In [19]:
red_bgr = [0, 0, 0]  # BGR for red
cap = cv2.VideoCapture(0)

In [22]:
while True:
    ret, frame = cap.read()

    hsvImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lowerLimit, upperLimit = get_limits(color=red_bgr)

    mask = cv2.inRange(hsvImage, lowerLimit, upperLimit)

    mask_ = Image.fromarray(mask)

    bbox = mask_.getbbox()

    if bbox is not None:
        x1, y1, x2, y2 = bbox

        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [35]:
import numpy as np
import cv2

def get_red_limits():
    # Define the lower and upper limits for red in HSV
    lower_red1 = np.array([0, 100, 100], dtype=np.uint8)    # Lower red (hue from 0 to 10)
    upper_red1 = np.array([10, 255, 255], dtype=np.uint8)

    lower_red2 = np.array([170, 100, 100], dtype=np.uint8)  # Upper red (hue from 170 to 180)
    upper_red2 = np.array([180, 255, 255], dtype=np.uint8)

    return (lower_red1, upper_red1), (lower_red2, upper_red2)

def detect_red(frame):
    # Convert the frame to HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the red color ranges
    (lower_red1, upper_red1), (lower_red2, upper_red2) = get_red_limits()

    # Create two masks to cover the red hue range
    mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)

    # Combine the two masks
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Find contours in the mask
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours, red_mask

def draw_rectangles(frame, contours):
    # Draw rectangles around detected contours
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small areas (adjust threshold as needed)
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw a green rectangle

# Open video capture (0 for webcam or provide video file path)
video_capture = cv2.VideoCapture(0)  # Use 'cv2.VideoCapture('path_to_video.mp4')' for video file

# Get frame width and height
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))

# Define the codec and create VideoWriter objects for each video
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for saving videos in .avi format

# Create VideoWriter objects for original, red-detected, and red-mask videos
original_writer = cv2.VideoWriter('original_output.mp4', fourcc, 20.0, (frame_width, frame_height))
red_detected_writer = cv2.VideoWriter('red_detected_output.mp4', fourcc, 20.0, (frame_width, frame_height))
red_mask_writer = cv2.VideoWriter('red_mask_output.mp4', fourcc, 20.0, (frame_width, frame_height), isColor=False)

while True:
    # Capture each frame from the video stream
    ret, frame = video_capture.read()

    if not ret:
        print("Failed to grab frame")
        break

    # Detect red color in the current frame
    contours, red_mask = detect_red(frame)

    # Draw rectangles around detected red areas
    draw_rectangles(frame, contours)

    # Write the frames to the respective video files
    original_writer.write(frame)            # Original video with rectangles
    red_detected_writer.write(frame)  # Red-detected video (now with rectangles)
    red_mask_writer.write(red_mask)          # Red mask video

    # Display the results
    cv2.imshow("Original Video", frame)
    cv2.imshow("Red Mask", red_mask)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects and close windows
video_capture.release()
original_writer.release()
red_detected_writer.release()
red_mask_writer.release()

cv2.destroyAllWindows()
